In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
iris = load_iris()
iris_X, iris_y = iris.data[:-1,:], iris.target[:-1]
iris_y= pd.get_dummies(iris_y).values
trainX, testX, trainY, testY = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)

In [5]:
# numFeatures is the number of features in our input data.
# In the iris dataset, this number is '4'.
numFeatures = trainX.shape[1]

# numLabels is the number of classes our data points can be in.
# In the iris dataset, this number is '3'.
numLabels = trainY.shape[1]


# Placeholders
# 'None' means TensorFlow shouldn't expect a fixed number in that dimension
X = tf.placeholder(tf.float32, [None, numFeatures]) # Iris has 4 features, so X is a tensor to hold our data.
yGold = tf.placeholder(tf.float32, [None, numLabels]) # This will be our correct answers matrix for 3 classes.

In [6]:
W = tf.Variable(tf.zeros([4, 3]))  # 4-dimensional input and  3 classes
b = tf.Variable(tf.zeros([3])) # 3-dimensional output [0,0,1],[0,1,0],[1,0,0]

In [7]:
#Randomly sample from a normal distribution with standard deviation .01

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))

In [8]:
# Three-component breakdown of the Logistic Regression equation.
# Note that these feed into each other.
apply_weights_OP = tf.matmul(X, weights, name="apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 
activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

# Number of Epochs in our training
numEpochs = 700

# Defining our learning rate iterations (decay)
learningRate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=trainX.shape[0],
                                          decay_rate= 0.95,
                                          staircase=True)


#Defining our cost function - Squared Mean Error
cost_OP = tf.nn.l2_loss(activation_OP-yGold, name="squared_error_cost")

#Defining our Gradient Descent
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

In [9]:
# Create a tensorflow session
sess = tf.Session()

# Initialize our weights and biases variables.
init_OP = tf.global_variables_initializer()

# Initialize all tensorflow variables
sess.run(init_OP)

In [10]:
# argmax(activation_OP, 1) returns the label with the most probability
# argmax(yGold, 1) is the correct label
correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(yGold,1))

# If every false prediction is 0 and every true prediction is 1, the average returns us the accuracy
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

# Summary op for regression output
activation_summary_OP = tf.summary.histogram("output", activation_OP)

# Summary op for accuracy
accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

# Summary op for cost
cost_summary_OP = tf.summary.scalar("cost", cost_OP)

# Summary ops to check how variables (W, b) are updating after each iteration
weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))
biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))

# Merge all summaries
merged = tf.summary.merge([activation_summary_OP, accuracy_summary_OP, cost_summary_OP, weightSummary, biasSummary])

# Summary writer
writer = tf.summary.FileWriter("summary_logs", sess.graph)

In [11]:
# Initialize reporting variables
cost = 0
diff = 1
epoch_values = []
accuracy_values = []
cost_values = []

# Training epochs
for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        # Run training step
        step = sess.run(training_OP, feed_dict={X: trainX, yGold: trainY})
        # Report occasional stats
        if i % 10 == 0:
            # Add epoch to epoch_values
            epoch_values.append(i)
            # Generate accuracy stats on test data
            train_accuracy, newCost = sess.run([accuracy_OP, cost_OP], feed_dict={X: trainX, yGold: trainY})
            # Add accuracy to live graphing variable
            accuracy_values.append(train_accuracy)
            # Add cost to live graphing variable
            cost_values.append(newCost)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))


# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={X: testX, 
                                                                yGold: testY})))

step 0, training accuracy 0.363636, cost 34.5643, change in cost 34.5643
step 10, training accuracy 0.555556, cost 30.1703, change in cost 4.39404
step 20, training accuracy 0.646465, cost 28.2048, change in cost 1.96551
step 30, training accuracy 0.646465, cost 26.5474, change in cost 1.65742
step 40, training accuracy 0.646465, cost 25.1714, change in cost 1.37595
step 50, training accuracy 0.646465, cost 24.0356, change in cost 1.13583
step 60, training accuracy 0.646465, cost 23.0961, change in cost 0.939486
step 70, training accuracy 0.646465, cost 22.3139, change in cost 0.782171
step 80, training accuracy 0.646465, cost 21.6569, change in cost 0.657022
step 90, training accuracy 0.646465, cost 21.0995, change in cost 0.557404
step 100, training accuracy 0.666667, cost 20.6218, change in cost 0.47773
step 110, training accuracy 0.666667, cost 20.2082, change in cost 0.41353
step 120, training accuracy 0.666667, cost 19.8469, change in cost 0.36138
step 130, training accuracy 0.66

step 310, training accuracy 0.79798, cost 16.7907, change in cost 0.0855713
step 320, training accuracy 0.79798, cost 16.7087, change in cost 0.082037
step 330, training accuracy 0.808081, cost 16.6299, change in cost 0.0787888
step 340, training accuracy 0.818182, cost 16.5541, change in cost 0.0758018
step 350, training accuracy 0.818182, cost 16.481, change in cost 0.0730381
step 360, training accuracy 0.838384, cost 16.4106, change in cost 0.0704765
step 370, training accuracy 0.838384, cost 16.3425, change in cost 0.0680981
step 380, training accuracy 0.838384, cost 16.2766, change in cost 0.0658817
step 390, training accuracy 0.838384, cost 16.2128, change in cost 0.0638123
step 400, training accuracy 0.848485, cost 16.1509, change in cost 0.0618744
step 410, training accuracy 0.848485, cost 16.0908, change in cost 0.0600586
step 420, training accuracy 0.858586, cost 16.0325, change in cost 0.0583515
step 430, training accuracy 0.858586, cost 15.9757, change in cost 0.0567427
ste

step 470, training accuracy 0.878788, cost 15.7631, change in cost 0.0511465
step 480, training accuracy 0.878788, cost 15.7132, change in cost 0.0499239
step 490, training accuracy 0.878788, cost 15.6645, change in cost 0.0487585
step 500, training accuracy 0.878788, cost 15.6168, change in cost 0.0476494
step 510, training accuracy 0.878788, cost 15.5702, change in cost 0.0465937
step 520, training accuracy 0.888889, cost 15.5246, change in cost 0.0455818
step 530, training accuracy 0.888889, cost 15.48, change in cost 0.0446148
step 540, training accuracy 0.89899, cost 15.4363, change in cost 0.0436907
step 550, training accuracy 0.89899, cost 15.3935, change in cost 0.0428038
step 560, training accuracy 0.89899, cost 15.3516, change in cost 0.0419521
step 570, training accuracy 0.89899, cost 15.3104, change in cost 0.0411339
step 580, training accuracy 0.89899, cost 15.2701, change in cost 0.040349
step 590, training accuracy 0.909091, cost 15.2305, change in cost 0.0395889


step 600, training accuracy 0.909091, cost 15.1916, change in cost 0.0388613
step 610, training accuracy 0.909091, cost 15.1535, change in cost 0.0381575
step 620, training accuracy 0.909091, cost 15.116, change in cost 0.0374794
step 630, training accuracy 0.909091, cost 15.0792, change in cost 0.0368233
step 640, training accuracy 0.909091, cost 15.043, change in cost 0.0361891
step 650, training accuracy 0.909091, cost 15.0074, change in cost 0.0355787
step 660, training accuracy 0.909091, cost 14.9724, change in cost 0.0349836
step 670, training accuracy 0.909091, cost 14.938, change in cost 0.0344086
step 680, training accuracy 0.909091, cost 14.9042, change in cost 0.0338526
step 690, training accuracy 0.909091, cost 14.8709, change in cost 0.0333128
final accuracy on test set: 0.9


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.plot([np.mean(cost_values[i-50:i]) for i in range(len(cost_values))])
plt.show()

